In [8]:
from pathlib import Path
import shutil
import subprocess
import os
import io
from io import StringIO
from contextlib import closing,redirect_stderr, redirect_stdout
import time
import json
from multiprocessing import Pool
import tqdm

In [10]:
mol_dir = Path("/dev/shm/cschlick/COD_processing/COD_finalized")
files = [file for file in mol_dir.glob("**/*") if file.suffix == ".mol"]

output_dir = Path("/dev/shm/cschlick/Elbow_on_COD")

In [11]:
def subtract_paths(p1,p2):
  return Path(*tuple(p for p in p1.parts if p not in p2.parts))

In [14]:
def worker(file,overwrite=True):
  relative_file = subtract_paths(file,mol_dir)
  folder = Path(relative_file.parent,file.stem)
  local_folder = Path(output_dir,folder)
  local_folder.mkdir(exist_ok=True,parents=True)
  if overwrite:
    existing_files = [f for f in local_folder.glob("**/*")]
    for f in existing_files:
      f.unlink()
  local_file = Path(local_folder,file.name)
  shutil.copyfile(file,local_file)
  os.chdir(local_folder)
  
  t0 = time.time()
  devnull = open(os.devnull, 'w')
  ret = subprocess.call("/net/cci-filer3/home/cschlick/Software/phenix/build/bin/phenix.elbow --quiet --overwrite"+str(local_file),shell=True,stdout=devnull, stderr=devnull)
  elapsed = time.time()-t0
  log = {"seconds":elapsed}
  logfile = Path(local_folder,"log.json")
  with logfile.open("w") as fh:
    json.dump(log,fh)

In [15]:
work = files
results = []
with closing(Pool(processes=64)) as pool:
  for result in tqdm.tqdm(pool.imap_unordered(worker, work), total=len(work)):
      results.append(result)
  pool.terminate()


  4%|██████▉                                                                                                                                                                | 2787/67598 [18:56<7:20:34,  2.45it/s]Process ForkPoolWorker-210:



KeyboardInterrupt: 

Process ForkPoolWorker-216:
Process ForkPoolWorker-222:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/net/cci/cschlick/miniconda3/envs/restraintsML/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/net/cci/cschlick/miniconda3/envs/restraintsML/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/net/cci/cschlick/miniconda3/envs/restraintsML/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/net/cci/cschlick/miniconda3/envs/restraintsML/lib/python3.9/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
  File "/net/cci/cschlick/miniconda3/envs/restraintsML/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/tmp/ipykernel_43064/1462717723.py", line 16, in worker
    ret = subprocess.call("/net/cci-filer3/home